<a href="https://www.kaggle.com/code/madhur321/solving-christmas-ornament-puzzles?scriptVersionId=158562002" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction

Every year at the North Pole, the elves decorate the workshop's grand tree with beautiful ornaments. These ornaments come in various geometric shapes, each with different colors and markings on their sides. However, some mischievous elves have mixed up the sides of these ornaments like unsolved puzzle cubes. We need to correct this before Santa notices!

In this notebook, we will tackle the challenge of solving these cube-like puzzles in the fewest moves possible. But these aren't your usual cubes - these puzzles come in a variety of geometric shapes. This might be easy for Santa, but it's a fun and challenging task for us data science enthusiasts.



# Approach

1. **Understanding the Problem**: We first need to understand the structure of these geometric puzzles and how their sides have been mixed up.

2. **Algorithm Design**: Next, we will design an algorithm to solve these puzzles in the fewest moves possible.

3. **Implementation**: We will then implement our algorithm and test it on a few puzzles to see how well it performs.

4. **Optimization**: Based on the performance of our algorithm, we might need to optimize it further to solve the puzzles more efficiently.

Let's get started and save Christmas!

In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import ast
import matplotlib.pyplot as plt
import seaborn as sns

from mpl_toolkits.mplot3d import Axes3D
from itertools import product, combinations
import plotly.graph_objects as go

import os
import fractions

In [20]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/santa-2023/sample_submission.csv
/kaggle/input/santa-2023/puzzles.csv
/kaggle/input/santa-2023/puzzle_info.csv


# Preprocessing

In [21]:
# Read the data
puzzle_info = pd.read_csv('/kaggle/input/santa-2023/puzzle_info.csv')
puzzles = pd.read_csv('/kaggle/input/santa-2023/puzzles.csv')
submission = pd.read_csv('/kaggle/input/santa-2023/sample_submission.csv')

In [32]:
puzzle=puzzles.merge(puzzle_info,on='puzzle_type',how='left')
#  puzzle is our dataframe and 'puzzle_type' is our column
puzzle[['puzzle', 'dimensions']] = puzzle['puzzle_type'].str.split('_', n=1, expand=True)
# puzzle is our dataframe and and 'solution_State' is our column
puzzle['solution_state'] = puzzle['solution_state'].str.split(';')
# # puzzle is our dataframe and and 'initial_state' is our column
puzzle['initial_state'] = puzzle['initial_state'].str.split(';')

In [4]:
puzzle

NameError: name 'puzzle' is not defined

In [ ]:
submission

# visualization


In [ ]:
## Creating a color dictionary 

In [8]:
# Define a list of colors
colors_list = ['red', 'blue', 'green', 'yellow', 'purple', 'orange', 'pink', 'brown', 'gray', 'black']

# Create a dictionary for 'A' to 'Z'
colors_AtoZ_upper = {chr(i+65): colors_list[i % len(colors_list)] for i in range(26)}

# Create a dictionary for 'a' to 'z'
colors_AtoZ_lower = {chr(i+97): colors_list[i % len(colors_list)] for i in range(26)}

# Create a dictionary for 'N0' to 'N225'
colors_N = {f'N{i}': colors_list[i % len(colors_list)] for i in range(40000)}

# Combine the dictionaries
colors = {**colors_AtoZ_upper, **colors_AtoZ_lower, **colors_N}

## For Cube shape

In [9]:
def draw_cube(ax, position, color):
    for s, e in combinations(np.array(list(product([0, 1], repeat=3))), 2):
        if np.sum(np.abs(s-e)) == 1:
            s = np.array(position) + np.array(s)
            e = np.array(position) + np.array(e)
            ax.plot3D(*zip(s, e), color=color)

def draw_cubes(row, colors):
    # Only execute if the puzzle is a cube
    if row['puzzle'] != 'cube':
        return

    # Get the dimensions of the cube
    dimensions = list(map(int, row['dimensions'].split('/')))

    # Get the initial and solution states
    initial_state = row['initial_state']
    solution_state = row['solution_state']
    id = row['id']

    fig = plt.figure(figsize=(12, 6))

    # Plot initial state
    ax1 = fig.add_subplot(121, projection='3d')
    for i, state in enumerate(initial_state):
        position = [i//dimensions[0], (i//dimensions[1])%dimensions[1], i%dimensions[2]]
        draw_cube(ax1, position, colors[state])
    ax1.set_title(f'Initial State of Puzzle {id}')

    # Plot solution state
    ax2 = fig.add_subplot(122, projection='3d')
    for i, state in enumerate(solution_state):
        position = [i//dimensions[0], (i//dimensions[1])%dimensions[1], i%dimensions[2]]
        draw_cube(ax2, position, colors[state])
    ax2.set_title(f'Solution State of Puzzle {id}')

    plt.show()


In [10]:
def plot_wreath_puzzles(df):
    # Loop over the rows of the dataframe
    for index, row in df.iterrows():
        # Check if the puzzle value is 'wreath'
        if row['puzzle'] == 'wreath':
            # Extract the values from the row
            initial_State = row['initial_state']
            solution_State = row['solution_state']
            dimensions = row['dimensions']
            id = row['id']

            # Split the initial_State and solution_State by ';'
            color_pattern_initial = initial_State
            color_pattern_solution = solution_State

            # Convert the dimensions to an integer
            num_points = np.prod([int(x) for x in dimensions.split('/')])

            # Define the theta and radius values
            theta = np.linspace(0, 2.*np.pi, num_points)
            r = np.abs(np.sin(5*theta))  # This creates a "wreath-like" pattern

            # Map the color pattern to actual colors
            colors_initial = [colors[color] for color in color_pattern_initial]
            colors_solution = [colors[color] for color in color_pattern_solution]

            # Create the polar plot
            fig, axs = plt.subplots(1, 2, subplot_kw=dict(polar=True), figsize=(12,6))

            # Plot initial state
            for i in range(num_points):
                axs[0].plot(theta[i], r[i], 'o', color=colors_initial[i % len(colors_initial)])
            axs[0].set_title(f'Initial State of Puzzle {id}')
            axs[0].set_yticklabels([])

            # Plot solution state
            for i in range(num_points):
                axs[1].plot(theta[i], r[i], 'o', color=colors_solution[i % len(colors_solution)])
            axs[1].set_title(f'Solution State of Puzzle {id}')
            axs[1].set_yticklabels([])

            # Show the plot
            plt.show()
        else:
            # Skip the row if the puzzle value is not 'wreath'
            pass

## For globe shape

In [11]:
def plot_globe_puzzles(df):
    # Loop over the rows of the dataframe
    for index, row in df.iterrows():
        # Check if the puzzle value is 'globe'
        if row['puzzle'] == 'globe':
            # Extract the values from the row
            initial_State = row['initial_state']
            solution_State = row['solution_state']
            dimensions = fractions.Fraction(row['dimensions'])  # Convert dimensions to a fraction
            id = row['id']

            # Map the color pattern to actual colors
            colors_initial = [colors[color] for color in initial_State]
            colors_solution = [colors[color] for color in solution_State]

            # Create a 3D plot
            fig, axs = plt.subplots(1, 2, subplot_kw={'projection':'3d'})

            # Create a sphere
            u = np.linspace(0, 2 * np.pi, 100)
            v = np.linspace(0, np.pi, 100)
            x = 10 * np.outer(np.cos(u), np.sin(v))
            y = 10 * np.outer(np.sin(u), np.sin(v))
            z = 10 * np.outer(np.ones(np.size(u)), np.cos(v))

            # Divide the sphere into sections according to the total number of colors
            total_colors = len(colors_initial)
            for i in range(total_colors):
                # Calculate the start and end indices for each section
                start = i*500//total_colors
                end = (i+1)*500//total_colors

                # Plot each section with the corresponding color
                axs[0].plot_surface(x[start:end, :], y[start:end, :], z[start:end, :], color=colors_initial[i])
                axs[1].plot_surface(x[start:end, :], y[start:end, :], z[start:end, :], color=colors_solution[i])

            axs[0].set_title(f'Initial State of Puzzle {id}')
            axs[1].set_title(f'Solution State of Puzzle {id}')
            
            plt.show()
        else:
            # Skip the row if the puzzle value is not 'globe'
            pass


In [ ]:
for i in range(len(puzzle)):
    draw_cubes(puzzle.iloc[i], colors)
    
plot_wreath_puzzles(puzzle)
plot_globe_puzzles(puzzle)

1. **Represent the puzzle as a graph**: Each state of the puzzle is a node in the graph. There's an edge between two nodes if you can get from one state to the other using a single allowed move or its inverse.

2. **Define the edge weights**: The weight of an edge could be defined as the number of moves required to get from one state to another. If a move is an inverse, you might define its weight as -1, for example.


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

# Assuming puzzle is your DataFrame and 'initial_state' and 'solution_state' are the columns with lists
for index, row in puzzle.iterrows():
    G = nx.DiGraph()  # Create a directed graph
    initial_state = row['initial_state']
    solution_state = row['solution_state']
    
    # Add nodes and edges with weights
    for i in range(len(initial_state)):
        G.add_node(initial_state[i])
        if i < len(initial_state) - 1:
            # The weight of an edge is defined as the number of moves required to get from one state to another
            G.add_edge(initial_state[i], initial_state[i+1], weight=1)
        if initial_state[i] in solution_state:
            G.add_node(solution_state[i])
            if i < len(solution_state) - 1:
                # If a move is an inverse, its weight is defined as -1
                G.add_edge(solution_state[i], solution_state[i+1], weight=-1 if solution_state[i+1] in initial_state else 1)
    
    plt.figure()
    pos = nx.spring_layout(G)  # positions for all nodes
    nx.draw_networkx_nodes(G, pos, node_size=700)
    nx.draw_networkx_edges(G, pos, edgelist=G.edges(), edge_color='black', arrows=True)
    nx.draw_networkx_labels(G, pos, font_size=20, font_family='sans-serif')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=nx.get_edge_attributes(G, 'weight'), label_pos=0.3, font_size=7)
    plt.title(f'Graph for row {index}')
    plt.show()


## Solution

In [37]:
# Read the data
puzzle_info = pd.read_csv('/kaggle/input/santa-2023/puzzle_info.csv')
puzzles = pd.read_csv('/kaggle/input/santa-2023/puzzles.csv')
submission = pd.read_csv('/kaggle/input/santa-2023/sample_submission.csv')

,id,puzzle_type,solution_state,initial_state,num_wildcards,allowed_moves,puzzle,dimensions
0,0,cube_2/2/2,"[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ...","[D, E, D, A, E, B, A, B, C, A, C, A, D, C, D, ...",0,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,...",cube,2/2/2
1,1,cube_2/2/2,"[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ...","[D, E, C, B, B, E, F, A, F, D, B, F, F, E, B, ...",0,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,...",cube,2/2/2
2,2,cube_2/2/2,"[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ...","[E, F, C, C, F, A, D, D, B, B, A, F, E, B, C, ...",0,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,...",cube,2/2/2
3,3,cube_2/2/2,"[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ...","[A, C, E, C, F, D, E, D, A, A, F, A, B, D, B, ...",0,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,...",cube,2/2/2
4,4,cube_2/2/2,"[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ...","[E, D, E, D, A, E, F, B, A, C, F, D, F, D, C, ...",0,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,...",cube,2/2/2
...,...,...,...,...,...,...,...,...
393,393,globe_3/33,"[A, A, A, A, A, A, C, C, C, C, C, C, E, E, E, ...","[D, D, L, A, P, E, R, U, U, C, S, R, J, B, E, ...",0,"{'r0': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,...",globe,3/33
394,394,globe_3/33,"[A, A, A, A, A, A, C, C, C, C, C, C, E, E, E, ...","[V, L, N, G, B, V, R, E, H, A, K, S, I, N, G, ...",0,"{'r0': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,...",globe,3/33
395,395,globe_3/33,"[N0, N1, N2, N3, N4, N5, N6, N7, N8, N9, N10, ...","[N12, N219, N227, N198, N4, N208, N214, N245, ...",0,"{'r0': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,...",globe,3/33
396,396,globe_8/25,"[A, A, A, A, A, D, D, D, D, D, G, G, G, G, G, ...","[V, P, F, L, P, X, O, A, J, b, V, Y, D, Y, C, ...",0,"{'r0': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,...",globe,8/25


In [22]:
puzzle=puzzles.merge(puzzle_info,on='puzzle_type',how='left')
#  puzzle is our dataframe and 'puzzle_type' is our column
puzzle[['puzzle', 'dimensions']] = puzzle['puzzle_type'].str.split('_', n=1, expand=True)
# puzzle is our dataframe and and 'solution_State' is our column
puzzle['solution_state'] = puzzle['solution_state'].str.split(';')
# # puzzle is our dataframe and and 'initial_state' is our column
puzzle['initial_state'] = puzzle['initial_state'].str.split(';')

In [40]:
# Function to convert numbers to ASCII alphabets
def convert_to_ascii(data):
    ascii_data = {}
    for key, values in data.items():
        ascii_values = [chr(value + 97) for value in values]  # 97 is the ASCII value of 'a'
        ascii_data[key] = ascii_values
    return ascii_data

In [50]:
puzzle['allowed_moves'] = puzzle['allowed_moves'].apply(ast.literal_eval).apply(convert_to_ascii)

In [52]:
type(puzzle['allowed_moves'])

pandas.core.series.Series

In [ ]:
# Convert the numbers to ASCII alphabets
ascii_data = convert_to_ascii(data)
print(ascii_data)

In [15]:
solution = df[['id','moves']]

In [18]:
solution.to_csv('output.csv', index=False)